# Multimodal Sentiment Analysis

###Overview
- This project performs sentiment analysis on MASAD, an open-source dataset containing ~38,000 text/image pairs.
- We preprocess the data into cleaned text and augmented images and then load the data.
- Our architecture is comprised of a transformer encoder, keyless attention to weight each modality, and a classification head.
- The test accuracy of our model is roughly 94% and our F1 score is also roughly 94%, comparable to the model used in the original MASAD paper.




---

###Instructions to Run Project
1. Download the following datasets: MASAD, MASAD_SUBSET
2. Create shortcut for both datasets in google drive

In [ ]:
!pip install tqdm
!pip install tensorboard

In [ ]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor
import re
import glob
import sys
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
import pickle
from transformers import get_linear_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f"model will run on {device}")

model will run on cuda


# Configuration parameters

In [ ]:
# Set configs for program (e.g. run w/ subset of data for faster processing & debugging)
use_data_subset = True
extract_text_bool = True
generate_clip_embeddings = True

In [ ]:
def set_max_cell_output_height(max_height=200):
    """ Helper function that visually limits the size of cell output blocks to reduce the amoutn of
        scrolling required for code block output
        Unfortuantely this cannot be set globally -- call in any functions which print long outputs
    """
    from IPython.display import Javascript
    display(Javascript(f'''google.colab.output.setIframeHeight(0, true, {{maxHeight: {max_height}}})'''))

# Preprocessing Data


###Accessing File Paths to Read & Process Data

In [ ]:
if use_data_subset:
  root_dir = '/content/drive/MyDrive/MASAD_SUBSET'
else:
  root_dir = '/content/drive/MyDrive/MASAD/'

###Preprocessing: Text only

In [ ]:
# Goal: Create a .csv with filepaths for both types and labels
# We need:
# 1. im paths
# 2. text paths
# 3. labels
image_paths = []
text_paths = []
labels = []

splits = ['train', 'test']
sentiments = ['negative', 'positive']

Step 1: Create a basic .csv where each row contains a valid image filepath in Google Drive, its corresponding valid text filepath, and the sentiment label of the pair.

In [ ]:
# Function to check if a file is a valid image or text file
# Attempt 1 at circumventing any data formatting issues while loading
def is_valid_file(file_path, extension):
    return os.path.isfile(file_path) and not os.path.basename(file_path).startswith('.') and file_path.lower().endswith(extension)

In [ ]:
def ensure_valid_data():

  # For dirs...
  for split in splits:
      for sentiment in sentiments:

          # Need to set paths for each pair given
          image_modality_path = os.path.join(root_dir, split, 'image', sentiment)
          text_modality_path = os.path.join(root_dir, split, 'text', sentiment)

          # Used for debugging missing ims in early stages
          # Checks and prints missing paths
          # This function may be responsible for bugs in training but unsure
          if not os.path.isdir(image_modality_path):
              print(f"Missing image modality directory: {image_modality_path}")
              continue
          if not os.path.isdir(text_modality_path):
              print(f"Missing text modality directory: {text_modality_path}")
              continue

          # Use glob to find all .jpg files
          # glob handles all of the logic here
          image_pattern = os.path.join(image_modality_path, '**', '*.jpg')
          found_images = glob.glob(image_pattern, recursive=True)

          for image_file in found_images:
              # Create a text path given image
              # Create valid path
              relative_image_path = os.path.relpath(image_file, os.path.join(root_dir, split, 'image', sentiment))
              text_file = os.path.join(root_dir, split, 'text', sentiment, relative_image_path)
              text_file = os.path.splitext(text_file)[0] + '.txt'

              # Check if the text file exists
              if is_valid_file(text_file, '.txt'):
                  image_paths.append(image_file)
                  text_paths.append(text_file)
                  labels.append(sentiment)
              else:
                  print(f"Missing text file for image: {image_file}")

  # Creat basic df
  data = {
      'image_path': image_paths,
      'text_path': text_paths,
      'label': labels
  }

  df = pd.DataFrame(data)


  # Save df
  target_dir = '/content/drive/MyDrive/computer_vision_final_project/'
  os.makedirs(target_dir, exist_ok=True)
  if use_data_subset:
      df_save_path = os.path.join(target_dir, 'MASAD_processed_subset.csv')
  else:
    df_save_path = os.path.join(target_dir, 'MASAD_processed.csv')
  df.to_csv(df_save_path, index=False)
  print(f"\nDataFrame saved at {df_save_path}")

Load basic filepaths .csv

In [ ]:
if use_data_subset:
    df_save_path = '/content/drive/MyDrive/computer_vision_final_project/MASAD_processed_subset.csv'
else:
  df_save_path = '/content/drive/MyDrive/computer_vision_final_project/MASAD_processed.csv'

# Load df
df = pd.read_csv(df_save_path)
print(df.head())

Step 2: Retrieve the raw text for each row and 'raw_text' col to df.

In [ ]:
tqdm.pandas()

def extract_text(text_file):
    try:
        with open(text_file, 'r', encoding='utf-8') as file:
            return file.read().strip()
    except Exception as e:
        # just return empty if not findable
        return ""

# Added progress bar with progress_apply because this takes forever
if extract_text_bool:
  print(f'value of extract text: {extract_text}')
  df['raw_text'] = df['text_path'].progress_apply(extract_text)
else:
  print(f'extract text if FALSE, so reading in the pickled df from MASAD_processed_clean_text.pkl')
  df = pd.read_pickle('/content/drive/MyDrive/computer_vision_final_project/MASAD_processed_clean_text.pkl')


# Display the first few entries
print(df[['text_path', 'raw_text']].head())

In [ ]:
if use_data_subset:
  save_path_raw = '/content/drive/MyDrive/computer_vision_final_project/MASAD_processed_raw_text_subset.pkl'
else:
  save_path_raw = '/content/drive/MyDrive/computer_vision_final_project/MASAD_processed_raw_text.pkl'

# save df
df.to_pickle(save_path_raw)
print(f"DataFrame with raw text saved at {save_path_raw}")

DataFrame with raw text saved at /content/drive/MyDrive/computer_vision_final_project/MASAD_processed_raw_text.pkl


In [ ]:
# Load the pickle file into a new df
loaded_df = pd.read_pickle(save_path_raw)

print(f"Loaded DataFrame shape: {loaded_df.shape}")
print(loaded_df.head())

Step 3: Clean raw text w/basic regex

In [ ]:
def clean_text(text):
    if not text:
        return ""
    # Remove all weird tags from MASAD
    text = re.sub(r'<tag>\S*', '', text)
    # remove urls/links
    text = re.sub(r'http\S+', '', text)

    # replace hashtag w/space
    text = re.sub(r'(#\s*)+', ' ', text)

    # Remove special chars and nums
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    # Return here if buggy
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Use tqdm progress in case takes as long as extract
df['clean_text'] = df['raw_text'].progress_apply(clean_text)

# Display the first few entries
print(df.head())

### Embedding Extraction
Transform images, pass them into clip, extract embeddings for this step of our pre-processing

In [ ]:
from transformers import CLIPImageProcessor, CLIPProcessor, CLIPImageProcessor, CLIPModel
import numpy as np
from transformers import CLIPImageProcessor, CLIPProcessor, CLIPTokenizerFast
from torchvision.transforms import v2
import torch.nn as nn
import torch.nn.functional as F
set_max_cell_output_height()
from PIL import Image, UnidentifiedImageError

def processed_im_and_text(df):
    im_list = []
    text_list = []
    im_paths = df['image_path'].tolist()
    indices_to_drop = []  # To keep track of corrupted image indices

    # Loop through image paths
    for i, img_path in enumerate(tqdm(im_paths, desc="Loading Images")):
        try:
            # Open and convert the image to RGB
            img = Image.open(img_path).convert("RGB")
            im_list.append(img)
            text_list.append(df.iloc[i]['clean_text'])
        except (UnidentifiedImageError, FileNotFoundError, OSError) as e:
            print(f"Warning: Skipping corrupted or unreadable image: {img_path} | Error: {e}")
            indices_to_drop.append(i)  # Mark this index for removal

    # Ensure image and text lists have the same length
    assert len(im_list) == len(text_list), "Mismatch between images and texts after processing."
    processed_inputs = None
    if len(im_list) != 0:
        processed_inputs = preprocess(text_list, im_list)
    return processed_inputs, indices_to_drop


def preprocess(texts, images):
    ''' Takes in a list of text and images, returns
    '''
    transforms = v2.Compose([
        v2.Resize(size=[224, 224]),
        v2.RandomResizedCrop(size=(224, 224), antialias=True),
        v2.RandomHorizontalFlip(p=0.2),
        v2.RandomRotation(90),
        v2.RandomZoomOut(p=0.3)])

    transformed_images = []
    for img in images:
        img = transforms(img)
        transformed_images.append(img)

    ## start of text token is T_CLS, start of image token is just CLS
    imageProcessor = CLIPImageProcessor(do_rescale=True, rescale_factor=1/255, do_normalize=True)
    tokenizer = CLIPTokenizerFast.from_pretrained('openai/clip-vit-base-patch16', bos_token='[T_CLS]')

    processor = CLIPProcessor(imageProcessor, tokenizer)
    inputs = processor(text=texts, images=transformed_images, truncation=True, return_tensors="pt", padding=True)
    return inputs

<IPython.core.display.Javascript object>

### data visualization functions to show original v.s. transformed images

In [ ]:
import matplotlib.pyplot as plt

def visualize_a_few_images(images, labels, display_label=False):
  # apply data augmentation to a few images
  transforms = v2.Compose([
    v2.Resize(size=[224, 224]),
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.2),
    v2.RandomRotation(90),
    v2.RandomZoomOut(p=0.3)])
  transformed_images = []
  for img in images:
      img = transforms(img)
      transformed_images.append(img)


  # plot each image and it's augmented version
  fig = plt.figure(figsize=(10, 7))
  plt.title('Sample Images and Transformations\n')
  plt.axis('off')

  for i in range(min(3, len(images))):
    im = images[i]
    t_im = transformed_images[i]

    plt.subplot(3, 2, 2*i+1)
    plt.imshow(im)
    plt.axis('off')  # Hide the axis labels
    if display_label:
      plt.title(f"Image {i+1} ({labels[i][0].upper()})") # 2*0+1=1, 2*1+1=3, 2*2+1=5
    else:
      plt.title(f"Image {i+1}") # 2*0+1=1, 2*1+1=3, 2*2+1=5

    plt.subplot(3, 2, 2*i+2) # 2*0+2=2, 2*1+2=4, 2*2+2=6
    plt.imshow(t_im)
    plt.axis('off')  # Hide the axis labels
    plt.title(f"Image {i+1} with Transformation")

  root_dir = '/content/drive/MyDrive/computer_vision_final_project/'
  plt.savefig(f'{root_dir}imgs_and_transform.png')
  plt.show()



In [ ]:
def extract_hidden_states(df, clip_model, batch_size=32):

    all_stacked_clip_embeddings = []
    labels = []
    total_samples = len(df)
    num_batches = (total_samples + batch_size - 1) // batch_size  # Ceiling division
    total_indices_dropped = 0
    for batch_idx in tqdm(range(num_batches), desc="Extracting Hidden States"):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(df))
        batch_inputs, indices_to_drop = processed_im_and_text(df[start_idx:end_idx])

        if indices_to_drop:
          df = df.drop(indices_to_drop).reset_index(drop=True)
          end_idx -= len(indices_to_drop)
          print(f"Dropped {len(indices_to_drop)} corrupted images. New DataFrame size: {df.shape}")
          total_indices_dropped += len(indices_to_drop)
        if batch_inputs == None:
          continue

        # Slice the batch data
        inputs = {
            'input_ids': batch_inputs['input_ids'].to(device),
            'attention_mask': batch_inputs['attention_mask'].to(device),
            'pixel_values': batch_inputs['pixel_values'].to(device)
        }
        inputs = {k: v.to(device) for k, v in inputs.items()}

        ## at this point, the batch inputs are the output of the clip processor
        with torch.no_grad():
            outputs = clip_model(**inputs)

        # get embeds instead of hidden states, if changing model to work with the whole sentence/image instead of tokenized version
        # Extract hidden states
        text_embeddings = outputs.text_embeds
        img_embeddings = outputs.image_embeds

        valid_batch_size = end_idx - start_idx - len(indices_to_drop)
        encoded_labels = process_labels(df.iloc[start_idx:end_idx]["label"].to_list())
        labels.extend(encoded_labels)
        combined_hidden_states = torch.stack((text_embeddings, img_embeddings), dim=1)

        # Move to CPU and detach to free GPU memory
        all_stacked_clip_embeddings.append(combined_hidden_states.cpu())
        total_examples = sum(tensor.shape[0] for tensor in all_stacked_clip_embeddings)
        print(f"Processed {total_examples} examples so far.Labels size: {len(df)}")

        # Optional: Clear CUDA cache to free memory
        torch.cuda.empty_cache()

    torch.save(all_stacked_clip_embeddings, 'stacked_clip_embeddings_list.pt') # this is necessary in case the code breaks on torch.cat due to shape issues
    # ^ if it does, we can just read in the embeddings without having to process them thru clip from scratch

    final_labels = torch.tensor(labels, dtype=torch.float)
    try:
      final_hidden_states = torch.cat(all_stacked_clip_embeddings, dim=0)
      return final_hidden_states, final_labels
    except Exception as e:
      print(f'ERROR: Encounterd the folowing error when trying to concatenate all the clip embeddings; \n{e}')
      print(f'You can debug this by using torch.load("stacked_clip_embeddings_list.pt") to avoid having to re-process things w/ clip')


def process_labels(labels):
    """ Goes through all labels and converts from a list of strings to a pytorch tensor, 0 if negative, 1 is positive
    """
    label_map = {"negative": 0, "positive": 1}  # Adjust as per your actual label names
    encoded_labels = [label_map[label] for label in labels]
    return torch.tensor(encoded_labels, dtype=torch.float)




In [ ]:
def save_model(model, save_path):
    """
    Saves the model's state_dict to the specified path.

    Args:
        model (nn.Module): The trained PyTorch model.
        save_path (str): The path where the model will be saved.
    """
    torch.save(model.state_dict(), save_path)
    print(f"Model weights saved to {save_path}")


# Main Pre-Processing Function

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/computer_vision_final_project/MASAD_processed_clean_text.pkl')

In [ ]:
target_rows_per_class = min(df['label'].value_counts())

df_small = df.groupby('label').sample(n=int(target_rows_per_class/1.5), random_state=42).reset_index(drop=True)
# Display the sizes of the original and scaled-down DataFrames
print(f"Original DataFrame size: {df.shape}")
print(f"Scaled-down DataFrame size: {df_small.shape}")
# (Optional) Verify the class distribution in the scaled-down DataFrame
print("\nClass distribution in scaled-down DataFrame:")
print(df_small['label'].value_counts())

Original DataFrame size: (38066, 5)
Scaled-down DataFrame size: (20136, 5)

Class distribution in scaled-down DataFrame:
label
negative    10068
positive    10068
Name: count, dtype: int64


### initialize cuda and clip model before pre-processing the data

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f"model will run on {device}")

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_model.eval()

for param in clip_model.parameters():
    param.requires_grad = False

model will run on cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

shapes into layer 1: 768, 512


Projection(
  (linear1): Linear(in_features=768, out_features=512, bias=False)
  (linear2): Linear(in_features=512, out_features=512, bias=False)
  (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (drop): Dropout(p=0.5, inplace=False)
)

In [ ]:
generate_clip_embeddings =True

In [ ]:
# Define the save directory on Google Drive
save_dir = '/content/drive/MyDrive/computer_vision_final_project/models/'
# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)
# Define the save path with a descriptive filename
embeddings_save_path = os.path.join(save_dir, 'mme_full')
labels_save_path = os.path.join(save_dir, 'labels_full')

if generate_clip_embeddings == True:
  mme,label_encodings = extract_hidden_states(df_small, clip_model) # mme is multimodal embedding

  print(f'successfully extracted the multimodal embeddings from CLIP with shape: {mme.shape}')
  print(f'label encodings shape: {label_encodings.shape}')

  # ☝🏽 modification 3:
  # the save model function is suitable for saving a model becaue it saves the model's state dict
  # but since our goal with the multimodal embeddings and labels is just to save them to a .pt file --
  # since they are state-less embeddings that don't have 'state_dict's, we just use torch.save() to
  # serialize them to a file
  torch.save(mme, embeddings_save_path)
  torch.save(label_encodings, labels_save_path)
  print(f"Embeddings saved at {embeddings_save_path}")
  print(f"Labels saved at {labels_save_path}")

else:
  print(f"generate_clip_embeddings is {generate_clip_embeddings}, so we'll just read in the embeddings and labels from their saved .pt files: \n{embeddings_save_path} and \n{labels_save_path}")
  mme = torch.load(embeddings_save_path)
  label_encodings = torch.load(labels_save_path)



num_batches 630
total_samples 20136


Extracting Hidden States:   0%|          | 0/630 [00:00<?, ?it/s]

end_idx 32



Extracting Hidden States:   0%|          | 1/630 [00:01<10:50,  1.03s/it]

clip_error True
text embeddings have shape: torch.Size([32, 512]) and img embeddings have shape: torch.Size([32, 512])
combined hidden states now has shape: torch.Size([32, 2, 512])
Processed 32 examples so far.Labels size: 20136
end_idx 64


...


Extracting Hidden States:  14%|█▍        | 91/630 [38:44<4:36:27, 30.77s/it]

clip_error True
text embeddings have shape: torch.Size([32, 512]) and img embeddings have shape: torch.Size([32, 512])
combined hidden states now has shape: torch.Size([32, 2, 512])
Processed 2910 examples so far.Labels size: 20134
end_idx 2944



Extracting Hidden States:  15%|█▍        | 92/630 [39:14<4:33:55, 30.55s/it]

clip_error True
text embeddings have shape: torch.Size([32, 512]) and img embeddings have shape: torch.Size([32, 512])
combined hidden states now has shape: torch.Size([32, 2, 512])
Processed 2942 examples so far.Labels size: 20134
end_idx 2976



Loading Images:  53%|█████▎    | 17/32 [00:16<00:13,  1.09it/s]

## Initialize Dataloader

In [ ]:
def process_labels(labels):
  """ Goes through all labels and converts from a list of strings to a pytorch tensor, 0 if negative, 1 is positive
  """
  label_map = {"negative": 0, "positive": 1}
  encoded_labels = [label_map[label] for label in labels]
  return torch.tensor(encoded_labels, dtype=torch.float)

In [ ]:
def extract_cls_emeddings(encoder_emeddings):
  text_cls_emeddings = encoder_emeddings[:, 0,:]
  image_cls_emeddings = encoder_emeddings[:, 77,:]
  return text_cls_emeddings, image_cls_emeddings

In [ ]:
from torch.utils.data import random_split, TensorDataset, DataLoader, Dataset

class MultiModalDataset(Dataset):
  def __init__(self, embeddings, sentiment_labels):
    self.embeddings = embeddings
    self.labels = sentiment_labels


  def __len__(self):
    """ Returns the number of samples in the dataset.
    """
    return int(len(self.labels))

  def __getitem__(self, idx):
    try:
      embedding = self.embeddings[idx]
      label = self.labels[idx]
      return embedding, label
    except Exception as e:
      print(f"Can't get sample")
      random_idx = torch.randint(0, len(self.embeddings), (1,)).item()
      return self.__getitem__(random_idx)

 \\

\\

\\

\\

\\

#Start Here

Load Data from saved

In [ ]:
subset_test = False

In [ ]:
if subset_test:
  embeddings_path = '/content/drive/MyDrive/MASAD_pivot/mme_1k.pt'
  labels_path = '/content/drive/MyDrive/MASAD_pivot/labels_1k.pt'
  print("Running on subset for testing.")
else:
  embeddings_path = '/content/drive/MyDrive/MASAD_pivot/mme_20k.pt'
  labels_path = '/content/drive/MyDrive/MASAD_pivot/labels_20k.pt'
  print("Running on full, balanced data for final results.")

Running on full, balanced data for final results.


Torch.load: https://pytorch.org/docs/stable/generated/torch.load.html \\
random_split: https://discuss.pytorch.org/t/how-to-split-dataset-into-test-and-validation-sets/33987/3

In [ ]:
embeddings = torch.load(embeddings_path)
labels = torch.load(labels_path)

print(embeddings.size())
print(len(labels))

<ipython-input-30-408fb416abce>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(embeddings_path)
<ipython-input-30-408fb416abce>:2: FutureWarning: Y

torch.Size([19982, 2, 512])
19982


Init dataset object for dataloader and get seeded random split

In [ ]:
# Using generator seed for consistency
generator = torch.Generator().manual_seed(42)

# Init dataset class for dataloaders later
dataset = MultiModalDataset(embeddings, labels)

split = 0.8

# get splits
train_split = int(split * dataset.__len__())
test_split = int(dataset.__len__() - train_split)

# Debugging size issues
print("Successful split" if dataset.__len__() == (train_split + test_split) else f"Split failed: train len = {train_split}, test len = {test_split}")

# Get train and test set
train_set, test_set = torch.utils.data.random_split(dataset, [train_split, test_split], generator=generator)

Successful split


In [ ]:
batch_size = 32

# DataLoaders for easy iter
train_loader = DataLoader(train_set, batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

## Data Visualization

In [ ]:
import pandas as pd
from PIL import ImageColor
df = pd.read_pickle(save_path_raw)
print(df.keys())
image_paths = df['image_path']
labels = df['label']
images = []

nvis = 4 # how many images to visualize
for i in range(nvis):
  im = Image.open(image_paths[i])
  images.append(im)

print(f'attempting to visualize {len(images)} with matplotlib')
visualize_a_few_images(images, labels[:nvis])

#Model Architecture

In [ ]:
class SequenceWiseTransformer(nn.Module):
    def __init__(self, d_model=512, nhead=8, num_layers=4, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, x):
      e = self.transformer_encoder(x)
      return e

In [ ]:
class KeylessAttention(nn.Module):
    def __init__(self, hidden_size):
        super(KeylessAttention, self).__init__()
        self.W = nn.Parameter(torch.empty(hidden_size))
        # Use Xavier rather than uniform for better inits
        nn.init.xavier_uniform_(self.W.unsqueeze(0))

    def forward(self, text_emb, im_emb):
        # Have pivoted to using entire sequence - level embeddings
        text_attn = torch.matmul(text_emb, self.W)
        im_attn = torch.matmul(im_emb, self.W)

        # Functional softmax is more numerically stable
        scores = torch.stack([text_attn, im_attn], dim=1)
        weights = F.softmax(scores, dim=1)
        w_text = weights[:, 0].unsqueeze(1)
        w_img = weights[:, 1].unsqueeze(1)

        # Aggregate the embeddings
        combined_emb = w_img * im_emb + w_text * text_emb
        return combined_emb

In [ ]:
class ClassificationHead(nn.Module):
    def __init__(self, d_model=512, num_classes=1):
        super().__init__()
        # hidden layer size to 128, then 32, then 1
        self.linear1 = nn.Linear(d_model, 128)
        self.linear2 = nn.Linear(128, 32)
        self.linear3 = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()

        # Tried normalization and it didn't really help
        # Adding dropout due to overfitting
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        pass_1 = self.linear1(x)
        basic_activation_1 = self.relu(pass_1)
        dropout_1 = self.dropout(basic_activation_1)
        pass_2 = self.linear2(dropout_1)
        basic_activation_2 = self.relu(pass_2)
        dropout_2 = self.dropout(basic_activation_2)
        x = self.linear3(dropout_2)
        return x

In [ ]:
class FullModel(nn.Module):
    def __init__(self, transformer, keyless_attention, classifier):
        super(FullModel, self).__init__()
        self.transformer = transformer
        self.keyless_attention = keyless_attention
        self.classifier = classifier

    def forward(self, x):
        encoded_embeddings = self.transformer(x) # Should be size [B, 2, D]
        combined_emb = self.keyless_attention(encoded_embeddings[:, 0, :], encoded_embeddings[:, 1, :])
        logits = self.classifier(combined_emb)
        return logits

#Train & Test Functions

Understanding BCEWithLogitsLoss vs CrossEntropyLoss: https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-pytorch-loss-functions.md

"This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability."

In [ ]:
def train(model, device, train_loader, optimizer, loss_fn, epoch,writer):
    model.train()
    running_loss = 0.0

    # Use tqdm for progress
    pbar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)

    # Do not detach
    for batch_idx, (data, label) in enumerate(pbar):
        data, label = data.to(device), label.to(device).unsqueeze(1).float()

        # Forward Pass
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, label)

        # Backward Pass & Optimize
        loss.backward()
        optimizer.step()

        # Increment loss and update pbar for tqdm
        running_loss += loss.item()
        pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/Train', avg_loss, epoch)


    print(f'\nEpoch {epoch} Training Loss: {avg_loss:.4f}')

In [ ]:
def test(model, device, test_loader, loss_fn, epoch,writer):
    print(f'*** RUNNING TEST FUNCTION ***')
    model.eval()
    test_loss = 0
    all_preds = []
    all_labels = []

    # Bug - removed "detach()"
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(tqdm(test_loader, desc="Testing", leave=False)):
            data, label = data.to(device), label.to(device).unsqueeze(1).float()

            # Get prediction; calculate test loss & # of correct predictions
            output = model(data)
            loss = loss_fn(output, label)
            test_loss += loss.item()

            probs = torch.sigmoid(output)
            preds = (probs >= 0.5).float()

            # Had to modify extended list due to format change
            all_preds.extend(preds.cpu().numpy().flatten().tolist())
            all_labels.extend(label.cpu().numpy().flatten().tolist())


    # Use sklearn to do eval comps for us :)
    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * sum([p==l for p,l in zip(all_preds, all_labels)]) / len(all_labels)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    roc = roc_auc_score(all_labels, all_preds)
    writer.add_scalar('Loss/Test', avg_loss, epoch)
    writer.add_scalar('Accuracy/Test', accuracy, epoch)


    print(f'Epoch {epoch}: Test Loss: {avg_loss:.4f}  |  Accuracy {accuracy:.2f}  |  Precision {precision:.2f}  |  Recall {recall:.2f}  |  F1 {f1:.2f}  |  F1 {f1:.2f}  |  ROC-AUC {roc:.2f}')

# Main

Init components

In [ ]:
transformer = SequenceWiseTransformer()
attention = KeylessAttention(hidden_size=512)
classifier = ClassificationHead()
model = FullModel(transformer, attention, classifier)
model.to(device)

In [ ]:
# Move all init params outside of train/test so they don't have to init on every train loop
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
# num_epochs = 20
# num_warmup_steps =
# total_steps = len(train_loader) *num_epochs
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=warmup_steps,  # Number of warmup steps
#     num_training_steps=total_steps  # Total number of training steps
# )

# scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='/content/drive/My Drive/tensorboard_logs/experiment1')

num_epochs = 20

for epoch in range(1, num_epochs+1):
    print(f'beginning epoch: {epoch}')
    train(model, device, train_loader, optimizer, loss_fn, epoch,writer)
    test(model, device, test_loader, loss_fn, epoch,writer)
    # scheduler.step()
writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/My Drive/tensorboard_logs"

#End Here

\\

\\

\\

\\

\\